Clean up if necessary

In [ ]:
import os
from shutil import rmtree
if os.path.isdir('mie2D_test'):
    rmtree('mie2D_test')


# Import and configuration

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.5)
import sys
sys.path.append('..')

if 'JCMPYTHON_CONFIG_FILE' in os.environ:
    del os.environ['JCMPYTHON_CONFIG_FILE']
import jcmpython as jpy

**jcmpython detected a configuration file:** 

In [ ]:
with open(jpy._config.config_file, 'r') as f:
    content = ''.join(f.readlines()[0:12])
print content+'...'

**We can print license or version info...**

In [ ]:
jpy.jcm_license_info()

**... but they are also stored in module attributes**

In [ ]:
print 'Version of JCMsuite:', jpy.__jcm_version__
print 'Version of jcmpython:', jpy.__version__

## Extensions

In [ ]:
print 'Known extensions:', jpy.extensions

In [ ]:
help(jpy.load_extension)

# Project management

**We specify the path to our project - absolute or relative to the configured project base**

In [ ]:
project = jpy.JCMProject('scattering/mie/mie2D')

**The project (.jcmp[t]) file is detected automatically, or can be specified**

In [ ]:
project.project_file_name

# Simulation sets

**We distinguish three types of input parameters:**
  - **`constants`**: can be of any type, but *not stored*
  - **`parameters`**: parameters that *do not* change the geometry
  - **`geometry`**: parameters that *do* change the geometry, i.e. belong to the `layout.jcmt`

**Let's specify 40 different radii**

In [ ]:
mie_keys = {'constants' :{},
            'parameters': {},
            'geometry': {'radius':np.linspace(0.3, 0.5, 40)}}

**We initialized a `SimulationSet`**

In [ ]:
simuset = jpy.SimulationSet(project, mie_keys,
                            storage_folder='mie2D_test',
                            storage_base=os.getcwd())

**The `storage_dir` now contains an `.h5` database file**

In [ ]:
os.listdir(simuset.storage_dir)

**We make a schedule:**

  1. all parameter combinations are determinded,
  2. simulations are sorted in a way to have minimal calls of JCMgeo
  3. database is checked for matching simulations which already have been done

In [ ]:
simuset.make_simulation_schedule()

**The store is empty**

In [ ]:
simuset.is_store_empty()

## Managing resources

**All configured resources are:**

In [ ]:
jpy.resources

**We can easily change the multiplicity and the number of threads**

In [ ]:
simuset.use_only_resources('localhost')
simuset.resources['localhost'].set_m_n(4,1)
print simuset.resources

## Computing geometries only

**The first simulation of our set has the following properties**

In [ ]:
sim = simuset.simulations[0]
print sim
print 'keys:', sim.keys

**Run jcm.geo**

In [ ]:
simuset.compute_geometry(sim)#, show=float('inf'))

The project's working directory now contains a grid file:

In [ ]:
'grid.jcm' in os.listdir(simuset.get_project_wdir())

## Running a single simulation

**We can solve a single simulation using our simulation set**

In [ ]:
results, logs = simuset.solve_single_simulation(sim)

**The results are stored in the simulation instance**

In [ ]:
print sim.logs['Out']

In [ ]:
print 'Exit code:', sim.exit_code
print 'Resource ID:', sim.resource_id
print 'Fieldbag file:', sim.fieldbag_file

## Result processing

**The default processing only extracts the computational costs**

In [ ]:
sim.process_results()

**The internal results dict now contains these results**

In [ ]:
sim._results_dict

**The status is updated**

In [ ]:
sim.status

**All post-processing results are stored as well**

In [ ]:
sim.jcm_results[1:]

**We can process the post processing results using a custom function**

In [ ]:
def read_scs(pp):
    results = {} #must be a dict
    results['SCS'] = pp[0]['ElectromagneticFieldEnergyFlux'][0][0].real
    return results

In [ ]:
sim.process_results(processing_func=read_scs, overwrite=True)

**'SCS' is now in the `_results_dict`:**

In [ ]:
sim._results_dict

## Running all simulations

**We can run and process all simulations in parallel using the simulation set**

In [ ]:
simuset.run(N=10, processing_func=read_scs)

## Data: pandas and the HDF5 store

**The store contains metadata**

In [ ]:
simuset.store

**E.g. for the used versions**

In [ ]:
simuset.store['version_data']

**We can get the data in the store as a pandas DataFrame**

In [ ]:
data = simuset.get_store_data().sort_values(by='radius')
data.info()

**pandas provides great methods to describe, query, save, plot, ...**

In [ ]:
interesting_columns = ['AccumulatedCPUTime', 'TotalMemory_GB', 'Unknowns', 'SCS']
data[interesting_columns].describe()

In [ ]:
data.plot(x='radius', y=interesting_columns, subplots=True, figsize=(8,12))
plt.show()

In [ ]:
FED_columns = ['FEDegree{}_Percentage'.format(i) for i in range(2,5)]
data.plot.area(x='radius', y=FED_columns, figsize=(8,6))
plt.ylim((0,100))
plt.show()

In [ ]:
data[FED_columns].mean(axis=0).plot.pie(autopct='%.2f', fontsize=16,
                                            figsize=(6, 6), title='Mean FE-Degrees')
plt.ylabel('')
plt.show()

## Adding data in a later run

**If the simulations failed or additional data needs to be added, the simulation set notices the existing data**

In [ ]:
extended_radii = np.append(np.linspace(0.5, 0.6, 40)[1:], np.linspace(0.3, 0.5, 40))
mie_keys_extended = {'constants' :{},
                     'parameters': {},
                     'geometry': {'radius':extended_radii}}

**Close the store ...**

In [ ]:
simuset.close_store()

**Schedule ...**

In [ ]:
simuset_ext = jpy.SimulationSet(project, mie_keys_extended,
                                storage_folder='mie2D_test',
                                storage_base=os.getcwd())
simuset_ext.make_simulation_schedule()

**Run the 39 remaining simulations and zip all working directories**

In [ ]:
simuset_ext.run(N=10, processing_func=read_scs, wdir_mode='zip')

**The storage folder contains the HDF5 store and a zip-archive**

In [ ]:
os.listdir(simuset_ext.storage_dir)

**Let's plot again**

In [ ]:
data_ext = simuset_ext.get_store_data().sort_values(by='radius')
data_ext.plot(x='radius', y='SCS', title='Results of the simulation')

**And write our data to a CSV**

In [ ]:
simuset_ext.write_store_data_to_file()

In [ ]:
os.listdir(simuset_ext.storage_dir)

In [ ]:
with open(os.path.join(simuset_ext.storage_dir, 'results.csv'), 'r') as f:
    content = ''.join(f.readlines()[0:12])
print content+'...'